In [ ]:
#| default_exp notebooks

# fastkaggle.core

> API details for fastkaggle notebooks to help manage notebooks/kernels

In [ ]:
#|hide
from nbdev.showdoc import *

In [ ]:
#|export
import os,json,subprocess, shutil
import re,ast
from string import punctuation

from fastcore.utils import *
from fastkaggle.core import *
from fastkaggle.competition import *
from nbdev.processors import nb_fmdict
from nbdev.read import read_nb
# from fastcore.all import *

In [ ]:
#|export
def nb2names(file):
    nb = read_nb(file)
    title = ast.literal_eval(nb_fmdict(nb)['title'])
    bad_chars = '|'.join(string.punctuation+string.whitespace)
    translator = title.maketrans(bad_chars,'-'*len(bad_chars))
    id = title.translate(translator)
    return title, id

In [ ]:
nb2names('05_notebooks.ipynb')

('fastkaggle.core', 'fastkaggle-core')

If you pass a list of space separated modules to `install`, they'll be installed if running on Kaggle.

In [ ]:
#|export
def nb_meta(file, cfg_path='.',private=True, gpu=False, internet=True):
    "Get the `dict` required for a kernel-metadata.json file"    
    print(cfg_path)
    cfg = get_config_values(cfg_path)
    competition = cfg['competition']
    cfg_datasets = [cfg['model_dataset_name'],cfg['libraries_dataset_name']]
    title,id = nb2names(file)    

    d = {
      "id": f"{get_username()}/{id}",
      "title": title,
      "code_file": file,
      "language": "python",
      "kernel_type": "notebook",
      "is_private": private,
      "enable_gpu": gpu,
      "enable_internet": internet,
      "keywords": [],
      "dataset_sources": cfg_datasets if cfg_datasets else [],
      "kernel_sources": []
    }
    if competition: d["competition_sources"] = [f"competitions/{competition}"]
    return d

In [ ]:
nb_meta('05_notebooks.ipynb')

.


{'id': 'isaacflath/fastkaggle-core',
 'title': 'fastkaggle.core',
 'code_file': '05_notebooks.ipynb',
 'language': 'python',
 'kernel_type': 'notebook',
 'is_private': True,
 'enable_gpu': False,
 'enable_internet': True,
 'keywords': [],
 'dataset_sources': ['models-titanic', 'libraries-titanic'],
 'kernel_sources': [],
 'competition_sources': ['competitions/titanic']}

In [ ]:
#|export
def push_notebook(file, path='.', private=True, gpu=False, internet=True):
    "Push notebook `file` to Kaggle Notebooks"
    meta = nb_meta(file=file, cfg_path=cfg_path, private=private, gpu=gpu, internet=internet)
    path = Path(path)
    nm = 'kernel-metadata.json'
    path.mkdir(exist_ok=True, parents=True)
    with open(path/nm, 'w') as f: json.dump(meta, f, indent=2)
    api = import_kaggle()
    api.kernels_push_cli(str(path))

## Export -

In [ ]:
#|hide
#|eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()